# Configurar TensorFlow
Importamos TensorFlow a nuestro programa para comenzar:

In [5]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.15.0


# Cargar un conjunto de datos
Cargamos y preparamos el conjunto de datos MNIST. Los valores de píxeles de las imágenes varían de 0 a 255. Escalamos estos valores a un rango de 0 a 1 dividiendo los valores por 255.0. Esto también convierte los datos de muestra de números enteros a números de punto flotante:

In [6]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Construye un modelo de aprendizaje automático
Construir un modelo tf.keras.Sequential:

In [7]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

**Sequential** es útil para apilar capas donde cada capa tiene un tensor de entrada y un tensor de salida. Las capas son funciones con una estructura matemática conocida que se pueden reutilizar y tienen variables entrenables. La mayoría de los modelos de TensorFlow se componen de capas. Este modelo utiliza las capas Flatten, Densey .Dropout

Para cada ejemplo, el modelo devuelve un vector de puntuaciones logits o log-odds , una para cada clase.

In [13]:
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.73370093,  0.03626865,  0.25953287, -0.6016903 ,  0.13557303,
        -0.6909094 , -0.5069855 , -0.2202093 , -0.07219306,  0.5148273 ]],
      dtype=float32)

La función tf.nn.softmax convierte estos logits en probabilidades para cada clase:

In [14]:
tf.nn.softmax(predictions).numpy()

array([[0.05325329, 0.11501117, 0.1437813 , 0.06076843, 0.1270186 ,
        0.05558155, 0.06680481, 0.08899242, 0.10318954, 0.1855989 ]],
      dtype=float32)

Nota: Es posible incorporar la función tf.nn.softmax a la función de activación para la última capa de la red. Si bien esto puede hacer que la salida del modelo sea más directamente interpretable, se desaconseja este enfoque ya que es imposible proporcionar un cálculo de pérdida exacto y numéricamente estable para todos los modelos cuando se utiliza una salida softmax.

Define una función de pérdida para entrenar usando losses.SparseCategoricalCrossentropy:

In [11]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

La función de pérdida toma un vector de valores de verdad fundamentales y un vector de logits y devuelve una pérdida escalar para cada ejemplo. Esta pérdida es igual a la probabilidad logarítmica negativa de la clase verdadera: la pérdida es cero si el modelo está seguro de la clase correcta.

Este modelo no entrenado proporciona probabilidades cercanas al azar (1/10 para cada clase), por lo que la pérdida inicial debería ser cercana a -tf.math.log(1/10) ~= 2.3.

In [15]:
loss_fn(y_train[:1], predictions).numpy()

2.889904

Antes de comenzar a entrenar, configure y compile el modelo Keras usando Model.compile. Establezca la clase optimizer en adam, establezca la pérdida con la función loss_fn que definiste anteriormente y especifica una métrica que se evaluará para el modelo estableciendo el parámetro metrics en accuracy.

In [16]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

# Entrena y evalúa tu modelo
Utiliza el método Model.fit para ajustar los parámetros de su modelo y minimizar la pérdida:

In [17]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 9s 4ms/step - loss: 0.2964 - accuracy: 0.9127
Epoch 2/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.1448 - accuracy: 0.9568
Epoch 3/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.1092 - accuracy: 0.9674
Epoch 4/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0874 - accuracy: 0.9734
Epoch 5/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0739 - accuracy: 0.9765


El método Model.evaluate verifica el rendimiento del modelo, generalmente en un conjunto de validación o de prueba.

In [18]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 1s - loss: 0.0742 - accuracy: 0.9777 - 750ms/epoch - 2ms/step


[0.07424662262201309, 0.9776999950408936]

El clasificador de imágenes ahora está entrenado con una precisión de ~98 % en este conjunto de datos.